In [1]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

In [ ]:
from google.colab import files
uploaded=files.upload()

In [3]:
df1=pd.read_excel("1.xlsx")
df2=pd.read_excel("2.xlsx")
df3=pd.read_excel("3.xlsx")

In [4]:
df = pd.concat([df1, df2, df3], ignore_index=True)


In [5]:
df = df[["questionText", "category_type", "answer", "categoriesParent"]]
df = df.apply(lambda col: col.str.strip() if col.dtype == "object" else col)
df['labels'] = df['categoriesParent'].apply(lambda x: [c.strip() for c in x.split(',')])

In [6]:
df.drop(columns=['categoriesParent'], inplace=True)

In [7]:
df = df.dropna(subset=["questionText", "category_type", "answer"])

# Optionally, reset the index after dropping rows (if needed)
df = df.reset_index(drop=True)

In [ ]:
df.head()

In [9]:
# Initialize the encoder
mlb = MultiLabelBinarizer()

# Fit and transform the categories to boolean values
encoded = mlb.fit_transform(df['labels'])

# Create a new DataFrame with the encoded categories
encoded_df = pd.DataFrame(encoded, columns=mlb.classes_, index=df.index)

# Concatenate with the original DataFrame (excluding 'categories' column)
df_encoded = pd.concat([df.drop('labels', axis=1), encoded_df], axis=1)

In [10]:
df=df_encoded

In [11]:
y = df.drop(columns=["questionText", "category_type", "answer"])

In [ ]:
minority_columns = y.sum()
below_300 = minority_columns[minority_columns < 300].index.tolist()

print(below_300)


In [13]:
from sklearn.utils import resample
import pandas as pd

def collect_and_pad_positive_rows(df, columns, target_n=300):
    """
    Collects all rows where each column == 1, and upscales to target_n rows per column.

    Args:
        df (pd.DataFrame): The full dataset
        columns (list): List of binary column names to check (e.g., ["x", "y"])
        target_n (int): Minimum number of rows to keep per class

    Returns:
        dict: Dictionary with keys = column names and values = padded DataFrames
    """
    results = {}

    for col in columns:
        subset = df[df[col] == 1].copy()
        n_rows = len(subset)

        if n_rows < target_n:
            # Upsample with replacement
            subset = resample(
                subset,
                replace=True,
                n_samples=target_n,
                random_state=42
            )

        results[col] = subset

    return results


In [14]:
columns_to_check = below_300
my_result = collect_and_pad_positive_rows(df, columns_to_check, target_n=300)


In [ ]:
for col, df_upsampled in my_result.items():
    count = df_upsampled[col].sum()
    total = len(df_upsampled)
    print(f"{col}: total rows = {total}, positive labels = {count}")


In [16]:
# Combine all upsampled DataFrames from your dict
padded_df = pd.concat(my_result.values()).reset_index(drop=True)


In [17]:
# 1. Remove original minority rows
df_filtered = df.copy()
for col in columns_to_check:
    df_filtered = df_filtered[df_filtered[col] != 1]

# 2. Combine the filtered df with the upsampled data
df_combined = pd.concat([df_filtered, padded_df]).reset_index(drop=True)


In [ ]:
df_combined[below_300].sum().sort_values()


In [ ]:
from google.colab import files

df_combined.to_csv("df_combined.csv", index=False)
files.download("df_combined.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>